# Module 4: File Systems & Module Architecture

### The Scenario

You've written a brilliant data processing script called `process.py`. It works perfectly when you run it directly. But when a colleague tries to `import process` to reuse your functions, the script immediately starts executing and overwrites their database. Then you try to share your code as a package, and nobody can install it because there's no `pyproject.toml`.

### The Goal

By the end of this module, you will understand:
- How Python distinguishes between **scripts** and **modules**
- How to organize code **professionally**
- How to configure packages with modern **TOML** files

### Example Files

This module uses example files in the `module4_examples/` directory. Open them alongside this notebook to follow along:

```markdown
module4_examples/
├── utils_bad.py          # Script without guard (bad)
├── utils_good.py         # Module with guard (good)
├── processor.py          # Professional entry point pattern
├── example.toml          # TOML syntax reference
├── pyproject.toml        # Complete pyproject.toml example
└── mypackage/            # Example package structure
    ├── __init__.py
    ├── core.py
    └── utils.py
```

---

## Lesson 1: Scripting vs. Module Execution

### The Problem

You wrote `utils.py` with some helper functions and a test at the bottom. When you run it directly, the test runs. But when someone imports it, the test runs too — corrupting their workflow.

### The "Aha!" Moment

Python sets a special variable `__name__` based on **how** the file is executed:

| Execution Method | `__name__` Value |
|-----------------|------------------|
| Run directly (`python utils.py`) | `"__main__"` |
| Imported (`import utils`) | `"utils"` (module name) |

### The Solution

The **`if __name__ == "__main__":`** guard prevents code from running on import.

**📂 Open side-by-side:** `module4_examples/utils_bad.py` and `module4_examples/utils_good.py`

In [1]:
# Understanding __name__

# In this notebook, __name__ is '__main__'
print(f"This notebook's __name__: {__name__}")

# When you import a module, its __name__ is the module name
import os
import json

print(f"os module __name__: {os.__name__}")
print(f"json module __name__: {json.__name__}")

This notebook's __name__: __main__
os module __name__: os
json module __name__: json


In [2]:
import sys

# Add examples directory to path
sys.path.insert(0, 'module4_examples')

print("Importing utils_bad (watch for unwanted execution):")
print("-" * 50)

Importing utils_bad (watch for unwanted execution):
--------------------------------------------------


In [3]:
# This will print test output - BAD!
import utils_bad

Running tests...
Test result: 60


In [4]:
print("Importing utils_good (should be silent):")
print("-" * 50)

import utils_good  # Silent - no test output

print("(No output = correct behavior!)")

Importing utils_good (should be silent):
--------------------------------------------------
(No output = correct behavior!)


In [5]:
# Both modules' functions work the same way
print(f"utils_bad.calculate_total([1,2,3]): {utils_bad.calculate_total([1,2,3])}")
print(f"utils_good.calculate_total([1,2,3]): {utils_good.calculate_total([1,2,3])}")

utils_bad.calculate_total([1,2,3]): 6
utils_good.calculate_total([1,2,3]): 6


### Professional Entry Point Pattern

Professional code uses a `main()` function as the entry point.

**📂 Open:** `module4_examples/processor.py`

Key patterns:
1. **Docstring at top** with usage instructions
2. **Functions are importable** - no side effects on import
3. **`main()` returns exit code** - 0 for success, non-zero for errors
4. **`sys.exit(main())`** in guard - propagates exit code to shell

In [6]:
# You can import and use the functions without running the script
import processor

# Use the functions directly
data = ["apple", "banana", "cherry"]
processed = processor.process_data(data)
print(f"Processed: {processed}")

Processed: ['APPLE', 'BANANA', 'CHERRY']


---

## Lesson 2: Python Project Structure

### The Problem

Your project has grown to 20 files scattered in one folder. Imports break constantly, tests can't find modules, and colleagues are confused.

### The "Aha!" Moment

Python projects follow a **standard structure**. Understanding **packages** (directories with `__init__.py`) and **modules** (`.py` files) is essential.

### Standard Project Layout

```
my_project/
├── pyproject.toml          # Project configuration
├── README.md               # Documentation
├── LICENSE                 # License file
├── src/                    # Source root (recommended)
│   └── my_package/         # Your actual package
│       ├── __init__.py     # Makes it a package
│       ├── core.py         # Core functionality
│       ├── utils.py        # Utilities
│       └── cli.py          # Command-line interface
├── tests/                  # Test directory
│   ├── __init__.py
│   ├── test_core.py
│   └── test_utils.py
└── docs/                   # Documentation
    └── index.md
```

**📂 Open:** `module4_examples/mypackage/` folder

### Understanding `__init__.py`

The `__init__.py` file serves multiple purposes:

| Purpose | Description |
|---------|-------------|
| **Package marker** | Marks directory as a Python package |
| **Initialization** | Runs when package is imported |
| **Public API** | Controls what `from pkg import *` exports via `__all__` |
| **Convenience imports** | Re-exports items for easier access |

**📂 Open:** `module4_examples/mypackage/__init__.py`

In [ ]:
# Different ways to import from a package

# Method 1: Import the package
import mypackage
print(f"Package version: {mypackage.__version__}")
print(f"Package __all__: {mypackage.__all__}")

In [ ]:
# Method 2: Import specific items (defined in __init__.py)
from mypackage import greet, calculate

print(f"greet('World'): {greet('World')}")
print(f"calculate(2, 3): {calculate(2, 3)}")

In [ ]:
# Method 3: Import from submodule directly
from mypackage.core import farewell

print(f"farewell('World'): {farewell('World')}")

# Note: farewell is NOT in __all__, so it won't be exported
# with "from mypackage import *", but can still be imported explicitly

In [ ]:
# Method 4: Import with alias
import mypackage as mp

print(f"mp.greet('Alias'): {mp.greet('Alias')}")

---

## Lesson 3: Understanding TOML

### The Problem

You want to share your package. But there are confusing files everywhere: `setup.py`, `setup.cfg`, `requirements.txt`, `MANIFEST.in`. Which do you need?

### The "Aha!" Moment

**`pyproject.toml`** is the modern, unified configuration file for Python projects. It replaces the chaos of multiple config files with one clear standard (PEP 517, 518, 621).

### TOML Basics

TOML (Tom's Obvious Minimal Language) is a configuration file format that's easy to read and write.

**📂 Open:** `module4_examples/example.toml`

### TOML Syntax Quick Reference

```toml
# Comments start with #

# Key-value pairs
title = "My Project"
version = "1.0.0"
enabled = true
count = 42

# Arrays
keywords = ["python", "example"]

# Tables (like dicts) - use [section]
[author]
name = "Jane Doe"
email = "jane@example.com"

# Nested tables
[database.credentials]
user = "admin"

# Inline tables
server = { ip = "10.0.0.1", role = "main" }

# Array of tables - use [[double brackets]]
[[plugins]]
name = "plugin-a"

[[plugins]]
name = "plugin-b"
```

In [ ]:
# Parsing TOML in Python
# Python 3.11+ has tomllib built-in
# For earlier versions: pip install tomli

try:
    import tomllib  # Python 3.11+
except ImportError:
    import tomli as tomllib  # Backport for older Python

# Parse the example TOML file
with open('module4_examples/example.toml', 'rb') as f:
    config = tomllib.load(f)

print(f"title: {config['title']}")
print(f"version: {config['version']}")
print(f"keywords: {config['keywords']}")
print(f"author.name: {config['author']['name']}")
print(f"plugins: {[p['name'] for p in config['plugins']]}")

---

## Lesson 4: The Complete `pyproject.toml`

**📂 Open:** `module4_examples/pyproject.toml`

### Anatomy of pyproject.toml

Three main sections:

| Section | Purpose |
|---------|--------|
| `[build-system]` | How to build the package |
| `[project]` | Package metadata (PEP 621) |
| `[tool.*]` | Tool-specific configuration |

### Section 1: `[build-system]`

Tells pip **how to build** your package.

```toml
[build-system]
requires = ["hatchling"]      # Packages needed for building
build-backend = "hatchling.build"  # The build tool
```

**Common build backends:**

| Backend | Best For |
|---------|----------|
| `setuptools` | Legacy projects, complex builds |
| `hatchling` | New projects, good defaults |
| `flit` | Pure Python packages |
| `poetry-core` | Poetry ecosystem users |

### Section 2: `[project]`

Package metadata for PyPI and pip.

```toml
[project]
name = "mypackage"
version = "0.1.0"
description = "A sample package"
requires-python = ">=3.10"

dependencies = [
    "requests>=2.28.0",
    "pydantic>=2.0",
]
```

### Section 3: Optional Dependencies

Install with: `pip install mypackage[dev]`

```toml
[project.optional-dependencies]
dev = [
    "pytest>=7.0",
    "black",
    "mypy",
]
docs = [
    "mkdocs",
    "mkdocs-material",
]
```

Install multiple groups: `pip install mypackage[dev,docs]`

### Section 4: CLI Entry Points

Creates executable commands when package is installed.

```toml
[project.scripts]
mypackage-cli = "mypackage.cli:main"
```

After `pip install`, running `mypackage-cli` calls `main()` in `mypackage/cli.py`

### Section 5: Tool Configurations

Replaces separate config files:

| Old File | New Section |
|----------|-------------|
| `pytest.ini` | `[tool.pytest.ini_options]` |
| `.black.toml` | `[tool.black]` |
| `mypy.ini` | `[tool.mypy]` |
| `.ruff.toml` | `[tool.ruff]` |

```toml
[tool.pytest.ini_options]
testpaths = ["tests"]
addopts = "-v --tb=short"

[tool.black]
line-length = 88

[tool.mypy]
python_version = "3.10"
warn_return_any = true
```

---

## Lesson 5: Building and Installing Your Package

### Development Workflow

```bash
# Install in development mode (editable)
pip install -e .

# Install with optional dependencies
pip install -e ".[dev]"
pip install -e ".[dev,docs]"
```

### Editable Install Explained

| Install Type | Behavior |
|--------------|----------|
| `pip install .` | Copies package to site-packages; changes require reinstall |
| `pip install -e .` | Creates link to source; changes take effect immediately |

**Always use `-e` during development!**

### Building for Distribution

```bash
# Install build tool
pip install build

# Build distribution files
python -m build

# Creates:
#   dist/mypackage-0.1.0.tar.gz      (source distribution)
#   dist/mypackage-0.1.0-py3-none-any.whl  (wheel)
```

### Publishing to PyPI

```bash
# Install twine
pip install twine

# Upload to Test PyPI first (recommended)
twine upload --repository testpypi dist/*

# Upload to PyPI
twine upload dist/*
```

---

## Summary

### Scripting vs. Module Execution

| Concept | Description |
|---------|-------------|
| `__name__` | `"__main__"` when run directly, module name when imported |
| Guard clause | `if __name__ == "__main__":` prevents code from running on import |
| Entry point | `main()` function + `sys.exit(main())` pattern |

### Project Structure

| Component | Purpose |
|-----------|--------|
| `src/package/` | Source code (src layout recommended) |
| `__init__.py` | Makes directory a package, controls public API |
| `__all__` | Controls `from package import *` |
| `tests/` | Test files |

### pyproject.toml

| Section | Purpose |
|---------|--------|
| `[build-system]` | Build backend configuration |
| `[project]` | Package metadata (name, version, dependencies) |
| `[project.scripts]` | CLI entry points |
| `[project.optional-dependencies]` | Extra dependency groups |
| `[tool.*]` | Tool-specific configuration |

---

**Next Module:** Memory, GIL, & Internal Performance

In [ ]:
# Cleanup: Remove module4_examples from sys.path
if 'module4_examples' in sys.path:
    sys.path.remove('module4_examples')

# Clean up imported modules
for mod in list(sys.modules.keys()):
    if mod.startswith(('utils_bad', 'utils_good', 'processor', 'mypackage')):
        del sys.modules[mod]

print("Module 4 complete!")